# Toronto Neighborhoods Clustering

In [40]:
import pandas as pd
import numpy as np

### Scrape the data from Wikipedia using Pandas

In [41]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

df = pd.read_html(link,header=0)[0]

In [42]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Droping rows which have no Borough assigned to them

In [28]:
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)

In [29]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Checking for duplicate Postal codes

In [32]:
pos = df["Postal code"]
df[pos.isin(pos[pos.duplicated()])].sort_values(by="Postal code")

,Postal code,Borough,Neighborhood


### Checking for rows having 'not assigned' value in the Neighborhood column

In [34]:
df[df['Neighborhood']=='Not assigned']

,Postal code,Borough,Neighborhood


### Final shape of the cleaned dataset

In [39]:
df.shape

(103, 3)